## Set-Up

In this code block we load the necessary python packages

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sve_gen as gen
import get_M as gm
import correlate as corr
import plot_correlation as pltcorr
import pca_on_correlations as pcaC
import plot_pc_map as pltPC
import time
import h5py

* set_id_cal: list of microstructure classes
* ns_cal: number of instantiations to generate for each class
* el: side dimension of the microstructures in pixels
* H: number of phases in the microstructures

In [ ]:
set_id_cal = ['bicrystal', 'gaussian', 'inclusion10', 'inclusion50','inclusion90']
ns_cal = [30, 30, 10, 10, 10]

# ns_cal = [30, 30, 30]
# set_id_cal = ['incl1', 'incl2', 'incl3']

el = 21
H = 2

wrt_file = 'log_%s.txt' % (time.strftime("%Y-%m-%d_h%Hm%M"))

## Microstructure Generation

Here we generate microstructure instantiations for each class

In [ ]:
f = h5py.File("spatial_stats.hdf5", 'w')
f.close()

gen.bicrystal_orthog(el, ns_cal[0], H, set_id_cal[0], wrt_file)
gen.improcess(el, ns_cal[1], H, set_id_cal[1], wrt_file)

vfrac = [.1]
gen.inclusion_red(el, ns_cal[2], H, set_id_cal[2], wrt_file, vfrac)
vfrac = [.5]
gen.inclusion_red(el, ns_cal[3], H, set_id_cal[3], wrt_file, vfrac)
vfrac = [.9]
gen.inclusion_red(el, ns_cal[4], H, set_id_cal[4], wrt_file, vfrac)

#gen.delta(el, ns_cal[3], H, set_id_cal[3], wrt_file)
#gen.bicrystal(el, ns_cal[4], H, set_id_cal[4], wrt_file)
#gen.inclusion(el, ns_cal[5], H, set_id_cal[5], wrt_file, vfrac)

# vfrac = [.075, .125]
# gen.inclusion_red(el, ns_cal[0], H, set_id_cal[0], wrt_file, vfrac)
# vfrac = [.100, .100]
# gen.inclusion_red(el, ns_cal[1], H, set_id_cal[1], wrt_file, vfrac)
# vfrac = [.125, .075]
# gen.inclusion_red(el, ns_cal[2], H, set_id_cal[2], wrt_file, vfrac)

## Calculate Microstructure Function

Here we find the microstructure function and perform the FFT algorithm on the results

In [ ]:
for ii in xrange(len(set_id_cal)):
    gm.get_M(el, ns_cal[ii], H, set_id_cal[ii], wrt_file)

## Calculate 2-Point Statistics

Here we calculate the full set of periodic 2-point spatial statistics for each microstructure instantiation

In [ ]:
"""Compute the periodic statistics for the microstructures"""
for ii in xrange(len(set_id_cal)):
    corr.correlate(el, ns_cal[ii], H, set_id_cal[ii], wrt_file)

Here we plot a microstructure instantiation and the associated 2-point statistics plot.
* set_num: the index associated with the microstructure class of interest
* sn: the index of the microstructure instantiation of interest
* iA: the index for the correlation of interest

In [ ]:
set_num = 1
sn = 0
iA = 0
pltcorr.pltcorr(el, ns_cal[set_num], set_id_cal[set_num], sn, iA)

## Perform Principal Component Analysis (PCA)

Here we perform dimensionality reduction on the spatial statistics through principal component analysis (PCA). The plot shows the percentage structural information captured by a certain number of pricipal components (PCs).

In [ ]:
pcaC.doPCA(el, ns_cal, H, set_id_cal, wrt_file)

Here we visualize the microstructure classes and thier associated instantiations in PC space. The spatial centers of each cluster are labeled by class.

In [ ]:
pcA = 0
pcB = 1
pltPC.pltmap(ns_cal, set_id_cal, pcA, pcB)